### 中醫醫案症狀詞抽取
- model: CRF
- 資料集：脈象
- 類別標註：OP
- 詞位標注: OSBIE
- 拆句: 以句點拆分
- 斷詞: 沒有做
- 詞性標註: NLTK
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(脈象)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "pulse" in keys:
        main = summary_json["pulse"]
        main_list = getTagListWithString(main)
        #print(main_list)
    #if "second" in keys:
    #    second = summary_json["second"]
    #    second_list = getTagListWithString(second)
        #print(second_list)
        
    #if len(second_list) > 0:
    #    main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [5]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 20030
Seconds required: 2.773

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 96068.660070
Feature norm: 1.000000
Error norm: 221637.781182
Active features: 20026
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.949

***** Iteration #2 *****
Loss: 83966.486132
Feature norm: 1.059831
Error norm: 168258.021887
Active features: 19919
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.485

***** Iteration #3 *****
Loss: 62153.147031
Feature norm: 1.248066
Error norm: 64075.127582
Active features: 17300
Line search trials: 1
Line search step: 1.000000
Seconds required f

***** Iteration #41 *****
Loss: 3653.747207
Feature norm: 23.139651
Error norm: 1344.324718
Active features: 4877
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.492

***** Iteration #42 *****
Loss: 3608.987212
Feature norm: 23.996082
Error norm: 466.715926
Active features: 4869
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.483

***** Iteration #43 *****
Loss: 3557.563004
Feature norm: 25.425439
Error norm: 356.168167
Active features: 4825
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.484

***** Iteration #44 *****
Loss: 3511.863007
Feature norm: 26.760648
Error norm: 602.474943
Active features: 4733
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.484

***** Iteration #45 *****
Loss: 3473.691541
Feature norm: 28.110267
Error norm: 824.011624
Active features: 4629
Line search trials: 1
Line search step: 1.000000
Seconds requi

***** Iteration #83 *****
Loss: 3175.445701
Feature norm: 42.415166
Error norm: 260.166630
Active features: 3259
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.981

***** Iteration #84 *****
Loss: 3173.465045
Feature norm: 42.522524
Error norm: 298.584920
Active features: 3243
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.485

***** Iteration #85 *****
Loss: 3172.212641
Feature norm: 42.673895
Error norm: 340.771988
Active features: 3241
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.969

***** Iteration #86 *****
Loss: 3170.970720
Feature norm: 42.713356
Error norm: 330.872972
Active features: 3243
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.972

***** Iteration #87 *****
Loss: 3170.270366
Feature norm: 42.878414
Error norm: 461.923026
Active features: 3231
Line search trials: 2
Line search step: 0.500000
Seconds requir

***** Iteration #128 *****
Loss: 3129.121795
Feature norm: 46.186958
Error norm: 228.076901
Active features: 2998
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.497

***** Iteration #129 *****
Loss: 3128.988223
Feature norm: 46.261503
Error norm: 424.287664
Active features: 3003
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.486

***** Iteration #130 *****
Loss: 3128.050035
Feature norm: 46.326334
Error norm: 207.257511
Active features: 3003
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.487

***** Iteration #131 *****
Loss: 3127.966585
Feature norm: 46.405436
Error norm: 446.002007
Active features: 3002
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #132 *****
Loss: 3126.974059
Feature norm: 46.454756
Error norm: 201.382730
Active features: 3000
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #169 *****
Loss: 3106.390990
Feature norm: 47.799893
Error norm: 204.199133
Active features: 2766
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.974

***** Iteration #170 *****
Loss: 3105.715139
Feature norm: 47.808982
Error norm: 259.044890
Active features: 2766
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.487

***** Iteration #171 *****
Loss: 3105.112192
Feature norm: 47.829359
Error norm: 155.150574
Active features: 2760
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.965

***** Iteration #172 *****
Loss: 3104.931289
Feature norm: 47.848636
Error norm: 452.737827
Active features: 2753
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #173 *****
Loss: 3104.064914
Feature norm: 47.912990
Error norm: 311.187663
Active features: 2754
Line search trials: 1
Line search step: 1.000000
Seconds r

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

心 (O)
率 (O)
8 (O)
2 (O)
次 (O)
/ (O)
分 (O)


In [12]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       1.00      1.00      1.00    317033
        B-S       0.70      0.81      0.75       596
        I-S       0.69      0.80      0.74       690
        E-S       0.70      0.81      0.75       594

avg / total       1.00      1.00      1.00    318913

